In [2]:
# Dependencies 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import scipy.stats as stats
from scipy.stats import linregress
from scipy.spatial import KDTree

# Impor the OpenWeatherMap API key
from api_keys import weather_api_key

# Import citipy to determine the cities based on latitude and longitude
#pip install citipy
#from citipy import citipy

In [3]:
# Empty list for holding the latitude and longitude combinations
lat_lngs = []

# Empty list for holding the cities names
cities = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = list(zip(lats, lngs))

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name

    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# combine cities and lattitudes and longitudes to one list to be easily
# converted as dataframe and easily call latitude and longitude for url
city_lat_long = list(zip(cities,lat_lngs))

# Print the city count to confirm sufficient count
print(f"Number of cities in the list: {len(cities)}")

NameError: name 'citipy' is not defined

In [ ]:
# generate the url to call the openweather API
#url = "https://api.openweathermap.org/data/3.0/onecall?lat="
url = "http://api.openweathermap.org/data/2.5/weather?"

# Define an empty list to fetch the weather data for each city
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0
    
    # Create endpoint URL with each city
    city_url = f"{url}appid={weather_api_key}&q={city}&units=imperial"

    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        
        # Parse out latitude, longitude, max temp, humidity, cloudiness, wind speed, country, and date
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_date = city_weather["dt"]

        # Append the City information into city_data list
        city_data.append({"City": city,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass

    # pause to avoid rate limiting
    time.sleep(1)

# Indicate that Data Loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [ ]:
# Convert the cities weather data into a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)

# Show Record Count
city_data_df.count()

In [ ]:
# Display sample data
city_data_df.head()

In [ ]:
# Export the City_Data into a csv
city_data_df.to_csv("output_data/cities.csv", index_label="City_ID")

In [ ]:
# Read saved data
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

In [ ]:
# Build scatter plot for latitude vs. temperature
plt.scatter(
    city_data_df['Lat'],
    city_data_df['Max Temp'],
    c='green',
    edgecolors='black',
)

# Incorporate the other graph properties
# change the label of the axes so it is readable and not abbreviated
plt.xlabel('Latitude')
plt.ylabel('Max Temperature (°F)')
plt.title('Latitude vs. Temperature')
plt.grid(True) 

# Save the figure
plt.savefig("output_data/Fig1.png")

# Show plot
plt.show()

In [ ]:
# Build the scatter plots for latitude vs. humidity
plt.scatter(
    city_data_df['Lat'],
    city_data_df['Humidity'],
    c='orange',
    edgecolors='black',
)

# Incorporate the other graph properties
# change the label of the axes so it is readable and not abbreviated
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.title('Latitude vs. Humidity')
plt.grid(True) 

# Save the figure
plt.savefig("output_data/Fig2.png")

# Show plot
plt.show()

In [ ]:
# Build the scatter plots for latitude vs. cloudiness
plt.scatter(
    city_data_df['Lat'],
    city_data_df['Cloudiness'],
    c='purple',
    edgecolors='black',
)

# Incorporate the other graph properties
# change the label of the axes so it is readable and not abbreviated
plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')
plt.title('Latitude vs. Cloudiness')
plt.grid(True) 

# Save the figure
plt.savefig("output_data/Fig3.png")

# Show plot
plt.show()

In [ ]:
# Build the scatter plots for latitude vs. wind speed
plt.scatter(
    city_data_df['Lat'],
    city_data_df['Wind Speed'],
    c='blue',
    edgecolors='black',
)

# Incorporate the other graph properties
# change the label of the axes so it is readable and not abbreviated
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')
plt.title('Latitude vs. Wind Speed')
plt.grid(True) 

# Save the figure
plt.savefig("output_data/Fig4.png")

# Show plot
plt.show()

In [ ]:
# Define a function to create Linear Regression plots
def linear_regression(x_values, y_values):
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
    regress_values = x_values * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    stdev = np.std(y_values)
    sem = stats.sem(y_values)
    # print linear regression calculations and slope formula to confirm accuracy
    print(
        f'\n'
        f'The slope of the regression is: {slope}.\n' 
        f'The inercept of the regression is: {intercept}.\n' 
        f'The rvalue of the regression is: {rvalue}.\n' 
        f'The pvalue of the regression is: {pvalue}.\n' 
        f'The Standard Deviation is: {stdev}.\n' 
        f'The Standard Error is: {stderr}.\n'
        f'The Standard Error of the mean is: {sem}.\n'
        f'The regression line equation is: {line_eq}.\n'
    )
    return regress_values, line_eq

In [ ]:
# Create a DataFrame with the Northern Hemisphere data (Latitude >= 0)
northern_hemi_df = city_data_df.loc[pd.to_numeric(city_data_df["Lat"]) >= 0, :]

# Display sample data
northern_hemi_df.head()

In [ ]:
# Create a DataFrame with the Southern Hemisphere data (Latitude < 0)
southern_hemi_df = city_data_df.loc[pd.to_numeric(city_data_df["Lat"]) <= 0, :]

# Display sample data
southern_hemi_df.head()

In [ ]:
# Linear regression on Northern Hemisphere of Temp vs. Latitude
# using the previously define linear regression function
regress_values, line_eq = linear_regression(northern_hemi_df["Lat"],northern_hemi_df["Max Temp"])

# create scatter plot with this data
plt.scatter(
    northern_hemi_df['Lat'],
    northern_hemi_df['Max Temp'],
    c='green',
    edgecolors='black',
)

# Incorporate the other graph properties
# change the label of the axes so it is readable and not abbreviated
plt.xlabel('Latitude')
plt.ylabel('Max Temperature (°F)')
plt.title('Latitude vs. Temperature | Northern Hemisphere')
plt.plot(northern_hemi_df["Lat"],regress_values,"r-")
plt.annotate(line_eq,(5.8,0.8),fontsize=15,color="red")
plt.grid(True) 

# Save the figure
plt.savefig("output_data/Fig5.png")

# Show plot
plt.show()

In [ ]:
# Linear regression on Southern Hemisphere
# using the previously define linear regression function
regress_values, line_eq = linear_regression(southern_hemi_df["Lat"],southern_hemi_df["Max Temp"])

# create scatter plot with this data
plt.scatter(
    southern_hemi_df['Lat'],
    southern_hemi_df['Max Temp'],
    c='green',
    edgecolors='black',
)

# Incorporate the other graph properties
# change the label of the axes so it is readable and not abbreviated
plt.xlabel('Latitude')
plt.ylabel('Max Temperature (°F)')
plt.title('Latitude vs. Temperature | Southern Hemisphere')
plt.plot(southern_hemi_df["Lat"],regress_values,"r-")
plt.annotate(line_eq,(-25,40),fontsize=15,color="red")
plt.grid(True) 

# Save the figure
plt.savefig("output_data/Fig6.png")

# Show plot
plt.show()

In [ ]:
# Northern Hemisphere Humidity vs. Latitude Regression Plot
# using the previously define linear regression function
regress_values, line_eq = linear_regression(northern_hemi_df["Lat"],northern_hemi_df["Humidity"])

# create scatter plot with this data
plt.scatter(
    northern_hemi_df['Lat'],
    northern_hemi_df['Humidity'],
    c='orange',
    edgecolors='black',
)

# Incorporate the other graph properties
# change the label of the axes so it is readable and not abbreviated
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.title('Latitude vs. Humidity | Northern Hemisphere')
plt.plot(northern_hemi_df["Lat"],regress_values,"r-")
plt.annotate(line_eq,(46,20),fontsize=15,color="red")
plt.grid(True) 

# Save the figure
plt.savefig("output_data/Fig7.png")

# Show plot
plt.show()

In [ ]:
# Southern Hemisphere
# using the previously define linear regression function
regress_values, line_eq = linear_regression(southern_hemi_df["Lat"],southern_hemi_df["Humidity"])

# create scatter plot with this data
plt.scatter(
    southern_hemi_df['Lat'],
    southern_hemi_df['Humidity'],
    c='orange',
    edgecolors='black',
)

# Incorporate the other graph properties
# change the label of the axes so it is readable and not abbreviated
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.title('Latitude vs. Humidity | Southern Hemisphere')
plt.plot(southern_hemi_df["Lat"],regress_values,"r-")
plt.annotate(line_eq,(-55,10),fontsize=15,color="red")
plt.grid(True) 

# Save the figure
plt.savefig("output_data/Fig8.png")

# Show plot
plt.show()

In [ ]:
# Northern Hemisphere 
# using the previously define linear regression function
regress_values, line_eq = linear_regression(northern_hemi_df["Lat"],northern_hemi_df["Cloudiness"])

# create scatter plot with this data
plt.scatter(
    northern_hemi_df['Lat'],
    northern_hemi_df['Cloudiness'],
    c='purple',
    edgecolors='black',
)

# Incorporate the other graph properties
# change the label of the axes so it is readable and not abbreviated
plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')
plt.title('Latitude vs. Cloudiness | Northern Hemisphere')
plt.plot(northern_hemi_df["Lat"],regress_values,"r-")
plt.annotate(line_eq,(46,46),fontsize=15,color="red")
plt.grid(True) 

# Save the figure
plt.savefig("output_data/Fig9.png")

# Show plot
plt.show()

In [ ]:
# Southern Hemisphere
# using the previously define linear regression function
regress_values, line_eq = linear_regression(southern_hemi_df["Lat"],southern_hemi_df["Cloudiness"])

# create scatter plot with this data
plt.scatter(
    southern_hemi_df['Lat'],
    southern_hemi_df['Cloudiness'],
    c='purple',
    edgecolors='black',
)

# Incorporate the other graph properties
# change the label of the axes so it is readable and not abbreviated
plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')
plt.title('Latitude vs. Cloudiness | Southern Hemisphere')
plt.plot(southern_hemi_df["Lat"],regress_values,"r-")
plt.annotate(line_eq,(-55,24),fontsize=10,color="red")
plt.grid(True) 

# Save the figure
plt.savefig("output_data/Fig10.png")

# Show plot
plt.show()

In [ ]:
# Northern Hemisphere
# using the previously define linear regression function
regress_values, line_eq = linear_regression(northern_hemi_df["Lat"],northern_hemi_df["Wind Speed"])

# create scatter plot with this data
plt.scatter(
    northern_hemi_df['Lat'],
    northern_hemi_df['Wind Speed'],
    c='blue',
    edgecolors='black',
)

# Incorporate the other graph properties
# change the label of the axes so it is readable and not abbreviated
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')
plt.title('Latitude vs. Wind Speed | Northern Hemisphere')
plt.plot(northern_hemi_df["Lat"],regress_values,"r-")
plt.annotate(line_eq,(46,46),fontsize=15,color="red")
plt.grid(True) 

# Save the figure
plt.savefig("output_data/Fig11.png")

# Show plot
plt.show()

In [1]:
# Southern Hemisphere
# using the previously define linear regression function
regress_values, line_eq = linear_regression(southern_hemi_df["Lat"],southern_hemi_df["Wind Speed"])

# create scatter plot with this data
plt.scatter(
    southern_hemi_df['Lat'],
    southern_hemi_df['Wind Speed'],
    c='blue',
    edgecolors='black',
)

# Incorporate the other graph properties
# change the label of the axes so it is readable and not abbreviated
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')
plt.title('Latitude vs. Wind Speed | Southern Hemisphere')
plt.plot(southern_hemi_df["Lat"],regress_values,"r-")
plt.annotate(line_eq,(46,46),fontsize=15,color="red")
plt.grid(True) 

# Save the figure
plt.savefig("output_data/Fig12.png")

# Show plot
plt.show()

NameError: name 'linear_regression' is not defined